# Parte 3

Projeto Final do Turing de NLP para os Trainees - Parte 3

Essa última parte do projeto se trata de explorarmos uma outra abordagem: o Word2Vec. Para isso, vamos passar o nosso dataset pré-processado e treinaremos o modelo para depois analisarmos os seus resultados (a partir de métodos como most_similar e doesnt_match)

In [1]:
#imports das bibliotecas para o nosso código
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
stop_words = set(stopwords.words('english')) 
from gensim.models.phrases import Phrases, Phraser

In [2]:
#lemos o dataset
df = pd.read_csv("IMDB Dataset.csv")

In [3]:
def strip_html_tags(texto):
    soup = BeautifulSoup(texto, "html.parser")
    texto_tratado = soup.get_text(separator=" ")
    return texto_tratado

In [4]:
def trata_review(review):
    
    review_2 = []
    for word in review:
        word_ = str(word)
        word_ = word_.lower() 
        review_2.append(word_)

    return review_2

In [5]:
def tira_stopwords(review):
    
    filtered_sentence = [] 
    for word in review:
        if word not in stop_words: 
            filtered_sentence.append(word) 
            
    return filtered_sentence

In [6]:
porter = PorterStemmer()

In [7]:
#igual a função da parte 1 de pré-processamento
def pre_processamentos(review):
    #primeiro vamos retirar as tags de html
    review = strip_html_tags(review)
    
    #selecionamos as palavras sem os números
    review = re.findall(r'[a-z,A-Z]\w+',review)
    
    #das palavras vamos retirar os espaços
    review = trata_review(review)
    
    #agora vamos tirar os stopwords
    review = tira_stopwords(review)
    
    #Stemização das palavras   
    for i in range(len(review)):
        review[i] = porter.stem(review[i])
                                          
    
    return review

In [8]:
#aplicamos o pré-processamento
df["review_tratado"] = df["review"].apply(lambda x: pre_processamentos(x))

In [14]:
#para facilitar a referenciação
texto = df["review_tratado"]

## Detectar bigramas
Vamos passar o nosso texto para identificar bigramas e coloquei que a palavra precisa aparecer pelo menos 20 vezes para ser considerada um bigrama:

In [72]:
phrases = Phrases(texto, min_count=2, threshold=20)

In [73]:
bigram = Phraser(phrases)

Estamos com o nosso modelo já detectando bigramas e depois de já ver alguns que ele identificou, mostrarei como que ele reconhece

In [74]:
print(bigram['there is a spoiler alert'.split()])

['there', 'is', 'a', 'spoiler_alert']


In [75]:
print(bigram['that film has an incredible special effect'.split()])

['that', 'film', 'has', 'an', 'incredible', 'special_effect']


In [76]:
#aplicando o bigrama para todo o nosso texto
sentences = bigram[texto]

## Treinar o modelo

In [77]:
#imports das bibliotecas
import multiprocessing
from gensim.models import Word2Vec

Para treinar o nosso modelo, precisamos passar alguns parâmetros:
* size = dimensionalidade dos word vectors
* sg = o algoritmo de treino
* min_count = ignora todas as palavras com frequência total menores que esse valor
* window = distância da palavra atual e da que queremos prever dentro de uma sentença

In [78]:
w2v = Word2Vec(min_count = 3, size = 100, window = 2, sg = 1)

In [79]:
#construção da tabela de vocabulário
w2v.build_vocab(sentences) 

Com o modelo preparado e a tabela de vocabulário construída, podemos treinar o modelo

In [80]:
w2v.train(sentences, total_examples = w2v.corpus_count, epochs = 15, report_delay = 1)

(75846426, 81930105)

## Análise do resultado 
Para analisarmos o resultado, vamos utilizar os seguintes métodos:
* most_similar
* similarity
* doesnt_match

### most_similar
Para esse método, podemos passar tanto "positive" como "negative" e pode ser um conjunto de palavras ao invés de uma só. Como retorno, recebemos as palvras mais ou menos próximas em relação a palavra(s) passada(s) como parâmetro(s).

In [82]:
w2v.wv.most_similar(positive=["film"])

[('movi', 0.905087947845459),
 ('tear_kali', 0.7367988228797913),
 ('inglouri_basterd', 0.7319300174713135),
 ('rescu_dawn', 0.7277275323867798),
 ('gujarati_theatr', 0.7251120209693909),
 ('irreproach', 0.723563551902771),
 ('casual_observ', 0.7228260040283203),
 ('strayer', 0.7208144664764404),
 ('chill_bone', 0.7205885648727417),
 ('incoher_narr', 0.7193603515625)]

In [83]:
w2v.wv.most_similar(negative=["film"])

[('roy', 0.051863331347703934),
 ('ra', 0.0025489740073680878),
 ('proprietor', -0.0045452117919921875),
 ('maven_recogn', -0.006327278912067413),
 ('lew', -0.0087735615670681),
 ('carson', -0.012495450675487518),
 ('burk', -0.012596890330314636),
 ('frederick', -0.014837939292192459),
 ('carolin', -0.01858524978160858),
 ('lesli', -0.020054414868354797)]

In [85]:
w2v.wv.most_similar(positive=["actor","best"])

[('versatil_actor', 0.6863749623298645),
 ('anthoni_asquith', 0.6685659885406494),
 ('second_consecut', 0.6678071022033691),
 ('john_hickam', 0.6673961877822876),
 ('michael_sopkiw', 0.6672186851501465),
 ('estonian', 0.6649237871170044),
 ('funniest_comedian', 0.6611506938934326),
 ('miguel_bardem', 0.6584398746490479),
 ('marg_simpson', 0.6552121043205261),
 ('hammiest', 0.6488018035888672)]

In [87]:
w2v.wv.most_similar(positive=["good","film"])

[('movi', 0.8408327698707581),
 ('chill_bone', 0.7950080633163452),
 ('sean_corinn', 0.7840585112571716),
 ('recomend_anyon', 0.7738760113716125),
 ('fiveson', 0.7710902690887451),
 ('adventur_hercul', 0.7694024443626404),
 ('squirm_induc', 0.7652233839035034),
 ('fiendishli', 0.764042854309082),
 ('senc', 0.7617179751396179),
 ('especiali', 0.7566429972648621)]

### similarity
Com esse método podemos ver o quão similar são duas palavras. Ela retorna o valor da similaridade entre elas como porcentagem:

In [88]:
w2v.wv.similarity("good","film")

0.44544148

In [89]:
w2v.wv.similarity("good","actor")

0.3484704

In [90]:
w2v.wv.similarity("actress","actor")

0.7305117

In [91]:
w2v.wv.similarity("good","bad")

0.69358337

In [93]:
w2v.wv.similarity("great","good")

0.7184862

In [96]:
w2v.wv.similarity("film","that")

0.42829686

In [97]:
w2v.wv.similarity("actor","good")

0.3484704

Vemos que palavras com características semelhantes apresentam uma maior similaridade e que, como esperado, a ordem em que as palavras são passadas não afeta no resultado

### doesnt_match
Esse método podemos passar uma lista de palavras e o modelo vai tentar acertar qual palavra que não tem relação com a lista

In [98]:
w2v.wv.doesnt_match(["actor","good","actress"])

/Users/kojimisumi/opt/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'good'

In [100]:
w2v.wv.doesnt_match(["perfect","good","actress","great"])

'actress'

In [101]:
w2v.wv.doesnt_match(["action","comedy","drama","romance","actor"])

'actor'

Pelos exemplos acima, vemos que o modelo conseguiu acertar na palavra que não pertencia a lista.

# Conclusão
Essa foi a última parte do Projeto Final de NLP para os Trainees. Foi muito interessanta ver como o modelo tenta pegar o contexto tanto para achar bigramas como para análise do modelo (como por exemplo "similarity" e "doesnt_match")para ter uma análise um pouco mais profunda sobre o texto (no caso, os reviews) ao invés de pegarem puramente as palavras separadamente. 

Gostei muito de terminar esse projeto! Foi muito desafiador e estou ansioso pelos próximos que virão!!